## Segmentação de Endometriose

Este notebook contem minha implementacao do codigo do artigo https://www.nature.com/articles/s41597-025-05623-3#article-info
Utilizamos o dataset publico: https://zenodo.org/records/15750762

é recomendado utilizar um Virtual Environment para baixar as dependencias presentes em requirements.txt

In [ ]:
#! venv env
! source env/bin/activate

#! pip install -r requirements.txt

: 

In [ ]:
import SimpleITK as sitk
import RAovSeg_tools as tools
import matplotlib.pyplot as plt
import numpy as np
import torch
import monai

: 

In [ ]:
# Load the image example
Img = sitk.ReadImage('./UTEndoMRI_example.nii.gz',sitk.sitkFloat64)
# Image resampling
Img = tools.ImgResample(Img, out_spacing=(0.35, 0.35, 6.0), out_size=(512,512,38), is_label=False, pad_value=0)
Img = sitk.GetArrayFromImage(Img)
# Image normalization
Img = tools.ImgNorm(Img,norm_type="percentile_clip",percentile_low=1,percentile_high=99)
# Image preprocessing
Img_preprossed = tools.preprocess_(Img, o1=0.24, o2=0.3)

# Plot and compare the original and processed image
plt.subplot(1,2,1)
plt.imshow(Img[18], cmap='gray')
plt.title("Original")
plt.subplot(1,2,2)
plt.imshow(Img_preprossed[18], cmap='gray')
plt.title("Preprocessed")
plt.show()

# Load the Segmentation an Model Prediction
Lb = sitk.ReadImage('./OvLabel.nii.gz',sitk.sitkInt32)
Lb = sitk.GetArrayFromImage(Lb)
Pred = sitk.ReadImage('./Prediction.nii.gz',sitk.sitkInt32)
Pred = sitk.GetArrayFromImage(Pred)
Pred_postprocessed = tools.postprocess_(Pred)

## Database
Neste bloco carregamos as imagens do dataset para realizar treinamento do modelo de classificacao e segmentacao

: 

In [ ]:
##########################################
### MODEL PIPELINE
##########################################
# Define the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

classification_model = monai.networks.nets.ResNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=2,
).to(device)

segmentation_model = monai.networks.nets.AttentionUnet(
    dimensions=3,
    in_channels=1,
    out_channels=2,
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=2,
).to(device)




In [ ]:
plt.figure(figsize=(9, 3))
plt.subplot(1, 3, 1)
plt.imshow(Lb[18], cmap='gray')
plt.title("Ground Truth")
plt.subplot(1, 3, 2)
plt.imshow(Pred[18], cmap='gray')
plt.title("Prediction")
plt.subplot(1, 3, 3)
plt.imshow(Pred_postprocessed[18], cmap='gray')
plt.title("Postprocessed")
plt.show()

# Dice calculation
dsc1 = tools.dsc_cal_np(Pred,Lb)
dsc2 = tools.dsc_cal_np(Pred_postprocessed,Lb)
print(f"The DSC between groundtruth and prediction is {dsc1}")
print(f"The DSC between groundtruth and postprocessed prediction is {dsc2}")